In [ ]:

"""
A producer P over a buffer list B is a set of workers P(W), a single job prod:(source, target) and holds a function 
ready signal s_p : B -> {0,1} indicating the buffers that need to be produced at the current time. 

it has states P.free, P.waiting, P.producing. 

A consumer C over that same buffer list B is also a set of workers and single job, and holds a function filled signal s_c : B->{0,1}
indicating the buffers that are filled and ready to consumed 

it has states C.free, C.waiting, C.consuming. 

a buffer b itself, is again a set/sequence of memory objects, (maybe some contigous chunk of memory)

and it has states b.empty, b.getting_produced, b.full, b.getting_consumed.  


okay, now let P be a producer, and C a consumer and B a list of buffers 

lets write some basic rules 

1. C.state(t+1) == C.waiting if  for all b in B, b.state(t) != b.full. 

2. P.state(t+1) == P.waiting if  for all b in B, b.state(t) != b.empty. 

3. The other is of course mutual exclisivity of states, a thing cannot be in two states at once 

4. P.state(t+1) = P.producing if for some b in B, b.state(t) = b.empty, in which case that b goes to b.state(t+1) = b.getting_produced
    and p.state(t) != p.producing. 
    
5  p.state(t+1) = P.free if P.state(t) = P.producing, in which case, the one b for which b.state(t) = b.getting_produced 
  now bas b.state(t+1) = b.full. 
  
6 C.state(t+1) = C.consuming if for some b in B, b.state(t) = b.full, in which case that b goes to b.state(t+1) = b.getting_consumed
   and C.state(t) != c.consuming 
7 C.state(t+1) = C.free if C.state(t) = C.consuming in which case the one b for which b.state(t) = b.getting_consumed 
now has state b.state(t_1) = b.empty. 



The init state is (P.free, C.free, bi = bi.empty for all bi in B)




"""

#### Correctness spec: 

we have a producer $P$ a consumer $C$ and a buffer set $B$. 
We have states 

$S(P) = \{p.f,s.w,p.pr\}$, $S(C) = \{c.f, c.w,c.cs\}$, $s(b) = \{e, gp, fl, gc\}$

###### System invariants: 

1. Mutual exclusivity, (implied by the set theory) any object can be in exactly one state at a given time. 
2. Producer lock: at any $t$, $S(P)_t = p.pr \iff !\exists b \in B: s(b)_t = gp$ 
3. Consumer lock: at any $t$  $S(C)_t = c.cs \iff !\exists b \in B: s(b)_t = gc$


##### Inital state: 

$(S(P)_0 = p.f, \ S(C)_0 = p.f, \  s(b) = e \  \forall b \in B$ 

##### tansitions 




In [ ]:
import numpy as np
class producer_consumer: 
  def __init__ (self, n_resources:int, n_producers:int, n_buffers:int, prod_job_latency:int, cons_job_latency:int, n_jobs): 
    """Here, we have n paralell resources of the same "size" in some sense (same number of threads say)
    we partition those into two parts, one part of producers, the other consumers. 
    we have a number of buffers, from which we can produce into and consume from, and we have some latency per resource 
    of both production and consumption, and it is obvoious that one resrouce handles the production/consumption of one job
    the goal is to compute consume(produce(job)) for n_jobs. 

    Args:
        n_resources (int): _description_
        n_producers (int): _description_
        n_buffers (int): _description_
        prod_job_latency (int): _description_
        cons_job_latency (int): _description_
        n_jobs (_type_): _description_
    """
    
    #we just creating as many states as there are atmost clock cycles which would just be (pl + cl + 1)*n_jobs
    self.pl = prod_job_latency 
    self.cl - cons_job_latency
    self.max_clocks = (self.pl + self.cl + 2)*n_jobs
    self.n_jobs = n_jobs 
    self.n_w = n_resources
    self.n_prod = n_producers 
    self.n_cons = self.n_w - self.n_prod
    self.n_buff = n_buffers
    
    self.producers_states = np.zeros((self.n_prod, self.max_clocks)).astype(int)
    self.consumers_states = np.zeros((self.n_cons,self.max_clocks)).astype(int)
    self.buffers_states = np.zeros((self.n_buff,n_jobs)).astype(int)
    
    self.producers_latency_points = np.zeros((self.n_prod)).astype(int)
    self.consumers_latency_points = np.zeros((self.n_cons)).astype(int)
    
    
    
    for i in range(1, self.max_clocks): 
      prev_empty_buffers = self.get_empty_buffers(self.buffers_states[i-1])
      prev_full_buffers = self.get_filled_buffers(self.buffers_states[i-1])
      prev_getting_produced_buffers = self.get_getting_produced_buffers(self.buffers_states[i-1])
      prev_getting_consumed_buffers = self.get_getting_consumed_buffers(self.buffers_states[i-1])
      prev_free_producers = self.get_free_producers(self.producers_states[i-1])
      prev_free_consumers = self.get_free_consumers(self.consumers_states[i-1])
      prev_waiting_producers = self.get_waiting_producers(self.producers_states[i-1])
      prev_waiting_consumers = self.get_waiting_consumers(self.consumers_states[i-1])
      prev_producing_producers = self.get_producing_producers(self.producers_states[i-1])
      prev_consuming_consumers = self.get_consuming_consumers(self.consumers_states[i-1])
      
  # --- Buffer State Getters ---
  # 0: empty
  # 1: getting_produced
  # 2: full
  # 3: getting_consumed

  def get_empty_buffers(self, buffers_state): 
    """Finds all buffers in the 'empty' (0) state."""
    return np.where(buffers_state == 0)

  def get_filled_buffers(self, buffers_state): 
    """Finds all buffers in the 'full' (1) state."""
    return np.where(buffers_state == 2)

  def get_getting_produced_buffers(self, buffers_state):
    """Finds all buffers in the 'getting_produced' (2) state."""
    return np.where(buffers_state == 1)

  def get_getting_consumed_buffers(self, buffers_state):
    """Finds all buffers in the 'getting_consumed' (3) state."""
    return np.where(buffers_state == 3)

  # --- Producer State Getters ---
  # 0: free
  # 1: waiting
  # 2: producing

  def get_free_producers(self, producers_state):
    """Finds all producers in the 'free' (0) state."""
    return np.where(producers_state == 0)

  def get_waiting_producers(self, producers_state):
    """Finds all producers in the 'waiting' (1) state."""
    return np.where(producers_state == 1)
    
  def get_producing_producers(self, producers_state): 
    """Finds all producers in the 'producing' (2) state."""
    return np.where(producers_state == 2)

  # --- Consumer State Getters ---
  # 0: free
  # 1: waiting
  # 2: consuming

  def get_free_consumers(self, consumers_state):
    """Finds all consumers in the 'free' (0) state."""
    return np.where(consumers_state == 0)

  def get_waiting_consumers(self, consumers_state):
    """Finds all consumers in the 'waiting' (1) state."""
    return np.where(consumers_state == 1)
    
  def get_consuming_consumers(self, consumers_state): 
    """Finds all consumers in the 'consuming' (2) state."""
    return np.where(consumers_state == 2)